In [85]:
import tensorflow as tf

In [86]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

x_train, x_test = x_train / 255., x_test / 255.

y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [87]:
learning_rate = 0.001
num_epochs = 50
batch_size = 256
display_step = 1
input_size = 784
hidden1_size = 256
hidden2_size = 256
output_size = 10

In [88]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

In [89]:
def random_normal_intializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [90]:
class ANN(tf.keras.Model):
  def __init__(self):
    super(ANN, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='relu',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

    self.output_layer = tf.keras.layers.Dense(output_size,
                                              activation=None,
                                              kernel_initializer=random_normal_intializer_with_stddev_1(),
                                              bias_initializer=random_normal_intializer_with_stddev_1())

  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    logits = self.output_layer(H2_output)

    return logits

In [91]:
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

In [92]:
optimizer = tf.optimizers.Adam(learning_rate)

In [93]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [94]:
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

ANN_model = ANN()

In [95]:
for epoch in range(num_epochs):
  average_loss = 0.
  total_batch = int(x_train.shape[0] / batch_size)
  for batch_x, batch_y in train_data:
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    average_loss += current_loss / total_batch

  if epoch%display_step == 0:
    print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

반복(Epoch): 1, 손실 함수(Loss): 274.093842
반복(Epoch): 2, 손실 함수(Loss): 61.347927
반복(Epoch): 3, 손실 함수(Loss): 40.275043
반복(Epoch): 4, 손실 함수(Loss): 29.740747
반복(Epoch): 5, 손실 함수(Loss): 23.037868
반복(Epoch): 6, 손실 함수(Loss): 18.249928
반복(Epoch): 7, 손실 함수(Loss): 14.758063
반복(Epoch): 8, 손실 함수(Loss): 12.001346
반복(Epoch): 9, 손실 함수(Loss): 9.790067
반복(Epoch): 10, 손실 함수(Loss): 8.102644
반복(Epoch): 11, 손실 함수(Loss): 6.537506
반복(Epoch): 12, 손실 함수(Loss): 5.369415
반복(Epoch): 13, 손실 함수(Loss): 4.384850
반복(Epoch): 14, 손실 함수(Loss): 3.553572
반복(Epoch): 15, 손실 함수(Loss): 2.935392
반복(Epoch): 16, 손실 함수(Loss): 2.387488
반복(Epoch): 17, 손실 함수(Loss): 1.893792
반복(Epoch): 18, 손실 함수(Loss): 1.485981
반복(Epoch): 19, 손실 함수(Loss): 1.103403
반복(Epoch): 20, 손실 함수(Loss): 0.885628
반복(Epoch): 21, 손실 함수(Loss): 0.683436
반복(Epoch): 22, 손실 함수(Loss): 0.552945
반복(Epoch): 23, 손실 함수(Loss): 0.381187
반복(Epoch): 24, 손실 함수(Loss): 0.317419
반복(Epoch): 25, 손실 함수(Loss): 0.239339
반복(Epoch): 26, 손실 함수(Loss): 0.194499
반복(Epoch): 27, 손실 함수(Loss): 0.120153
반

In [96]:
print("정확도(Accuracy): %f" % compute_accuracy(ANN_model(x_test), y_test))

정확도(Accuracy): 0.949200
